# Pre-trained DenseNet model in Pytorch [Transfer Learning]

### Goal: pytorch official 문서를 따라해보며 Pre-trained DenseNet 모델을 가져와서 feature-extract을 해본다.

## Reference
* https://pytorch.org/tutorials/beginner/finetuning_torchvision_models_tutorial.html
* 사용 데이터: pytorch official 문서에서 제공하는 bees/ants 구분 데이터

In [1]:
from __future__ import print_function
from __future__ import division
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
print("PyTorch Version: ",torch.__version__)
print("Torchvision Version: ",torchvision.__version__)

PyTorch Version:  1.2.0
Torchvision Version:  0.4.0


### 1. 기본 세팅
* ImageFolder를 사용할 수 있는 directory인지 확인해본다.

In [9]:
# data 구조는 ImageFolder 함수에 알맞게 되어있어야 함.
data_dir = "C:/Users/sbh0613/Desktop/CV/hymenoptera_data"

# [resnet, alexnet, vgg, squeezenet, densenet, inception] 중에서 하나를 선택
model_name = "densenet"

num_classes = 2

batch_size = 8

num_epochs = 15

# False면 whole model을 훈련하고 True면 일부 layer만 변경함.
feature_extract = True

### 2. train_model 함수 정의하기

In [10]:
def train_model(model, dataloaders, criterion, optimizer, num_epochs=25, is_inception=False):
    since = time.time()

    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        # Each epoch has a training and validation phase
        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                # track history if only in train
                with torch.set_grad_enabled(phase == 'train'):
                    # Get model outputs and calculate loss
                    # Special case for inception because in training it has an auxiliary output. In train
                    #   mode we calculate the loss by summing the final output and the auxiliary output
                    #   but in testing we only consider the final output.
                    if is_inception and phase == 'train':
                        # From https://discuss.pytorch.org/t/how-to-optimize-inception-model-with-auxiliary-classifiers/7958
                        outputs, aux_outputs = model(inputs)
                        loss1 = criterion(outputs, labels)
                        loss2 = criterion(aux_outputs, labels)
                        loss = loss1 + 0.4*loss2
                    else:
                        outputs = model(inputs)
                        loss = criterion(outputs, labels)

                    _, preds = torch.max(outputs, 1)

                    # backward + optimize only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            # deep copy the model
            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)

        print()

    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    model.load_state_dict(best_model_wts)
    return model, val_acc_history

### 3. 모델의 파라미터의 requires_grad attribute 정하기
* 우리가 위에서 feature exracting을 False라고 정했으면, 새롭게 initialized 된 layer의 wiehgt만 학습할 것이기 때문에 우선 전체적으로 False라고 지정.
* 이후에 새로운 layer을 만들면 자동으로 requires_grad는 True로 된다.

#### 참고 사항: fine-tuning과 feature_extract는 다른 것이다.
* fine-tuning은 layer의 일부를 변경하고, 그 모델을 첨부터 다시 train하는 것이다. 따라서 requies_grad는 모두 True이다.
* feature_exract는 layer의 일부를 변경하고 그 변경된 부분의 weight만 학습하는 것이다. 따라서 requies_grad는 새롭게 변경된 layer만 True이고 나머지는 모두 False이다.
* 여기서는 feature_extract을 할 것이므로 requies_grad를 False로 지정한다.

In [11]:
def set_parameter_requires_grad(model, feature_extracting):
    if feature_extracting:
        for param in model.parameters():
            param.requires_grad = False

### 4. pre-trained model 불러오기
* 여기서는 우선 binary image 분류이므로 가장 가벼운 모델인 densenet121을 사용

In [4]:
from torchvision import models
model = models.densenet121(pretrained=True)

Downloading: "https://download.pytorch.org/models/densenet121-a639ec97.pth" to C:\Users\sbh0613/.cache\torch\checkpoints\densenet121-a639ec97.pth
100%|█████████████████████████████████████████████████████████████████████████████| 30.8M/30.8M [00:13<00:00, 2.41MB/s]


In [6]:
print(model)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

)


마지막 layer가 (classifier): Linear(in_features=1024, out_features=1000, bias=True) 이다. 이를 아래와 같이 바꾼다.

In [7]:
model.classifier = nn.Linear(1024, num_classes)

pytorch에서는 이를 한번에 하는 initialize_model 함수를 제공한다.

In [12]:
def initialize_model(model_name, num_classes, feature_extract, use_pretrained=True):
    # Initialize these variables which will be set in this if statement. Each of these
    #   variables is model specific.
    model_ft = None
    input_size = 0

    if model_name == "resnet":
        """ Resnet18
        """
        model_ft = models.resnet18(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "alexnet":
        """ Alexnet
        """
        model_ft = models.alexnet(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "vgg":
        """ VGG11_bn
        """
        model_ft = models.vgg11_bn(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier[6].in_features
        model_ft.classifier[6] = nn.Linear(num_ftrs,num_classes)
        input_size = 224

    elif model_name == "squeezenet":
        """ Squeezenet
        """
        model_ft = models.squeezenet1_0(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        model_ft.classifier[1] = nn.Conv2d(512, num_classes, kernel_size=(1,1), stride=(1,1))
        model_ft.num_classes = num_classes
        input_size = 224

    elif model_name == "densenet":
        """ Densenet
        """
        model_ft = models.densenet121(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        num_ftrs = model_ft.classifier.in_features
        model_ft.classifier = nn.Linear(num_ftrs, num_classes)
        input_size = 224

    elif model_name == "inception":
        """ Inception v3
        Be careful, expects (299,299) sized images and has auxiliary output
        """
        model_ft = models.inception_v3(pretrained=use_pretrained)
        set_parameter_requires_grad(model_ft, feature_extract)
        # Handle the auxilary net
        num_ftrs = model_ft.AuxLogits.fc.in_features
        model_ft.AuxLogits.fc = nn.Linear(num_ftrs, num_classes)
        # Handle the primary net
        num_ftrs = model_ft.fc.in_features
        model_ft.fc = nn.Linear(num_ftrs,num_classes)
        input_size = 299

    else:
        print("Invalid model name, exiting...")
        exit()

    return model_ft, input_size

# Initialize the model for this run
model_ft, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=True)

# Print the model we just instantiated
print(model_ft)

DenseNet(
  (features): Sequential(
    (conv0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (norm0): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (relu0): ReLU(inplace=True)
    (pool0): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (denseblock1): _DenseBlock(
      (denselayer1): _DenseLayer(
        (norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu1): ReLU(inplace=True)
        (conv1): Conv2d(64, 128, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (norm2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu2): ReLU(inplace=True)
        (conv2): Conv2d(128, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      )
      (denselayer2): _DenseLayer(
        (norm1): BatchNorm2d(96, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu

)


### 5. 데이터 전처리 및 불러오기

In [13]:
# Data augmentation and normalization for training
# Just normalization for validation
data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(input_size),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.Resize(input_size),
        transforms.CenterCrop(input_size),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}

print("Initializing Datasets and Dataloaders...")

# Create training and validation datasets
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x]) for x in ['train', 'val']}
# Create training and validation dataloaders
dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=batch_size, shuffle=True, num_workers=4) for x in ['train', 'val']}

# Detect if we have a GPU available
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

Initializing Datasets and Dataloaders...


### 6. optimizer 만들기
* pretrained model을 로드하고, reshaping (우리가 원하는 layer을 initialized)하기 이전에 직접 feature_extrac을 False로 만들어줬었다.
* 그리고 initialized layer의 requires_grad는 우리가 initialize하면 자동으로 True가 됨을 앞서 언급했다.
* 따라서 requires_grad가 True인 파라미터만 optimize 되어야 한다.
* 이러한 파라미터만 뽑아서 optimize consructor의 input으로 넣을 것이다.
* optimizer는 Adam을 사용한다.

In [17]:
# Send the model to GPU
model_ft = model_ft.to(device)

# Gather the parameters to be optimized/updated in this run. If we are
#  finetuning we will be updating all parameters. However, if we are
#  doing feature extract method, we will only update the parameters
#  that we have just initialized, i.e. the parameters with requires_grad
#  is True.
params_to_update = model_ft.parameters()
print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_ft.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(params_to_update, lr=0.001)

Params to learn:
	 classifier.weight
	 classifier.bias


* 우리가 마지막의 classifier layer만 num_class가 2로 수정했었다.
* 위 결과를 보면 classifier의 weight, bias만 requires_grad가 자동으로 True가 되었음을 확인할 수 있다.

### 7. 훈련 & 테스트

In [18]:
# Setup the loss fxn
criterion = nn.CrossEntropyLoss()

# Train and evaluate
model_ft, hist = train_model(model_ft, dataloaders_dict, criterion, optimizer_ft, num_epochs=num_epochs, is_inception=(model_name=="inception"))

Epoch 0/14
----------
train Loss: 0.6498 Acc: 0.6393
val Loss: 0.5799 Acc: 0.6928

Epoch 1/14
----------
train Loss: 0.5648 Acc: 0.7418
val Loss: 0.5243 Acc: 0.7908

Epoch 2/14
----------
train Loss: 0.5074 Acc: 0.8033
val Loss: 0.4506 Acc: 0.8627

Epoch 3/14
----------
train Loss: 0.4848 Acc: 0.8115
val Loss: 0.4091 Acc: 0.8954

Epoch 4/14
----------
train Loss: 0.4260 Acc: 0.8566
val Loss: 0.3894 Acc: 0.8824

Epoch 5/14
----------
train Loss: 0.4292 Acc: 0.8402
val Loss: 0.3517 Acc: 0.9020

Epoch 6/14
----------
train Loss: 0.4216 Acc: 0.8607
val Loss: 0.3282 Acc: 0.9085

Epoch 7/14
----------
train Loss: 0.3831 Acc: 0.8893
val Loss: 0.3267 Acc: 0.8889

Epoch 8/14
----------
train Loss: 0.3794 Acc: 0.8648
val Loss: 0.3016 Acc: 0.9150

Epoch 9/14
----------
train Loss: 0.3685 Acc: 0.8607
val Loss: 0.2978 Acc: 0.9020

Epoch 10/14
----------
train Loss: 0.3670 Acc: 0.8320
val Loss: 0.2797 Acc: 0.9150

Epoch 11/14
----------
train Loss: 0.3363 Acc: 0.8852
val Loss: 0.2780 Acc: 0.9020

Ep